In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

## Bayesian Linear Regression 

In [ ]:
# build a basis function of degree p (0, 1, ..., p)
def phiGen(p):
    return lambda x: np.array([x**i for i in range(p+1)]).T

p = 3
phi = phiGen(p)

In [ ]:
x = np.linspace(-1,1)

plt.plot(x, phi(x));

plt.axhline(0, color='black')
plt.axvline(0, color='black')

## Compute $Cov(f(x_1), f(x_2))$:

In [ ]:
beta_cov = np.eye(p+1)
f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T))

plt.imshow(f_cov)
plt.colorbar()

## Sample $f$:

In [ ]:
f = scipy.stats.multivariate_normal.rvs(np.zeros(50), f_cov, size=10).T

plt.plot(x,f);
plt.savefig('figures/f_linreg_samples.pdf',bbox_inches='tight')

In [ ]:
# build a function distribution, with stability in covariance

def fdist(x, phi, beta_cov, sigma = 0.0):
    f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T)) + np.eye(x.shape[0])*sigma
    
    jit = 1e-6
    excp = None
    while jit<1e1:
        try:
            return scipy.stats.multivariate_normal(np.zeros(x.shape[0]), f_cov+jit*np.eye(x.shape[0]))
        except np.linalg.LinAlgError as e:
            jit = jit*10
            excp = e
    
    raise excp

# Training model on data

### sample data from prior:

In [ ]:
p = 4
phi = phiGen(p)

xsamp = np.random.choice(x, size=3, replace=False)
xsamp.sort()

fsamp = fdist(xsamp, phiGen(p), np.eye(p+1)).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

### compute conditional distribution:

$cov(f(x_2), f(x_1))$:

In [ ]:
k21 = np.dot(phi(x), np.dot(np.eye(p+1), phi(xsamp).T))

$cov(f(x_2), f(x_2))$:

In [ ]:
k22 = np.dot(phi(x), np.dot(np.eye(p+1), phi(x).T))

$cov(f(x_1), f(x_1))$

In [ ]:
k11 = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) 

# stabilize matrix
k11 += np.eye(xsamp.shape[0]) * 1e-9

conditional:

In [ ]:
mu_pred = np.dot(k21, np.dot(np.linalg.inv(k11), fsamp))
cov_pred = k22 - np.dot(k21, np.dot(np.linalg.inv(k11), k21.T))
# cov_pred[cov_pred<1e-9] = 1e-9
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

plt.savefig('figures/f_linreg_conditional_noiseless.pdf', bbox_inches='tight')

### Add observation noise:

In [ ]:
x = np.linspace(-1,1)

sigma = 5e-1
p = 4
phi = phiGen(p)
xsamp = np.random.uniform(x.min(), x.max(), size=10)
fsamp = fdist(xsamp, phiGen(p), np.eye(p+1), sigma=sigma).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

In [ ]:
k21 = np.dot(phiGen(p)(x), np.dot(np.eye(p+1), phiGen(p)(xsamp).T))
k22 = np.dot(phiGen(p)(x), np.dot(np.eye(p+1), phiGen(p)(x).T))
k11 = np.dot(phiGen(p)(xsamp), np.dot(np.eye(p+1), phiGen(p)(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(k21, np.dot(np.linalg.inv(k11), fsamp))
cov_pred = k22 - np.dot(k21, np.dot(np.linalg.inv(k11), k21.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

plt.savefig('figures/f_linreg_conditional_noiseful.pdf', bbox_inches='tight')

In [ ]:
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.plot(x, scipy.stats.multivariate_normal.rvs(mu_pred, cov_pred, size=10).T,alpha=.5);

plt.savefig('figures/f_linreg_conditional_noiseful_samples.pdf', bbox_inches='tight')

### what about predicting new observations?

In [ ]:
k22 += sigma*np.eye(x.shape[0])

In [ ]:
mu_pred = np.dot(k21, np.dot(np.linalg.inv(k11), fsamp))
cov_pred = k22 - np.dot(k21, np.dot(np.linalg.inv(k11), k21.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(x, scipy.stats.multivariate_normal.rvs(mu_pred, cov_pred, size=1).T,alpha=.5);
plt.scatter(xsamp, fsamp, c='k')

plt.savefig('figures/f_linreg_conditional_predictive.pdf', bbox_inches='tight')

## Kernels

## Radial basis function kernel

In [ ]:
x = np.linspace(-1,1, 101)[:,None]

In [ ]:
kern = GPy.kern.RBF(1, variance=1., lengthscale=1.)

In [ ]:
plt.figure(figsize=(12,6))

c = 10

plt.subplot2grid((c,c*2+2), (0,0), colspan=c, rowspan=c)
plt.plot(x[x.shape[0]/2,0] - x, kern.K(x)[x.shape[0]/2,:])

plt.subplot2grid((c,c*2+2), (0,c+1), colspan=c, rowspan=c)
plt.imshow(kern.K(x))

ax = plt.subplot2grid((c,c*2+2), (0,c*2+1), colspan=1, rowspan=c)
plt.colorbar(cax=ax)

In [ ]:
plt.figure(figsize=(10,6))
for i, ls in enumerate(10**np.arange(-1, 2, 1.)):
    k = GPy.kern.RBF(1, lengthscale=ls)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\ell = %.1lf$'%ls)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/lengthscale.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,6))
for i, var in enumerate(10**np.arange(-.5, 1, .5)):
    k = GPy.kern.RBF(1, variance=var)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\sigma^2 = %.1lf$'%var)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/variance.pdf',bbox_inches='tight')

### what about our linear basis data?

In [ ]:
# compute posterior using linear basis

kzx = np.dot(phiGen(p)(x[:,0]), np.dot(np.eye(p+1), phiGen(p)(xsamp).T))
kzz = np.dot(phiGen(p)(x[:,0]), np.dot(np.eye(p+1), phiGen(p)(x[:,0]).T))
kxx = np.dot(phiGen(p)(xsamp), np.dot(np.eye(p+1), phiGen(p)(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred, label='linear model')
plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)

# compute posterior using RBF kernel
kern = GPy.kern.RBF(1, variance=1, lengthscale=1)
kzx = kern.K(x, xsamp[:,None])
kzz = kern.K(x)
kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred, color='C1', label='RBF kernel')
plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1, color='C1')

plt.scatter(xsamp, fsamp, color='k')
plt.legend()

plt.savefig('figures/f_linreg_condition_rbf.pdf', bbox_inches='tight')

In [ ]:
for i, ls in enumerate(10**np.arange(-1, 2, 1.)):
    kern = GPy.kern.RBF(1, lengthscale=ls)

    kzx = kern.K(x, xsamp[:,None])
    kzz = kern.K(x)
    kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

    mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
    cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
    std_pred = np.sqrt(np.diag(cov_pred))

    plt.plot(x, mu_pred, color='C%d'%i, label='$\ell=%.1lf$'%ls)
    plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1, color='C%d'%i)

plt.scatter(xsamp, fsamp, color='k')
plt.legend()

plt.savefig('figures/f_linreg_condition_rbf_ls-vary.pdf', bbox_inches='tight')

### matern

In [ ]:
plt.figure(figsize=(10,6))
for i, ls in enumerate(10**np.arange(-1, 2, 1.)):
    k = GPy.kern.Matern32(1, lengthscale=ls)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\ell = %.1lf$'%ls)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/lengthscale-matern32.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,6))
for i, ls in enumerate(10**np.arange(-1, 2, 1.)):
    k = GPy.kern.Matern52(1, lengthscale=ls)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\ell = %.1lf$'%ls)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/lengthscale-matern52.pdf',bbox_inches='tight')

In [ ]:
# compute posterior using linear basis

kzx = np.dot(phiGen(p)(x[:,0]), np.dot(np.eye(p+1), phiGen(p)(xsamp).T))
kzz = np.dot(phiGen(p)(x[:,0]), np.dot(np.eye(p+1), phiGen(p)(x[:,0]).T))
kxx = np.dot(phiGen(p)(xsamp), np.dot(np.eye(p+1), phiGen(p)(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

# plt.plot(x, mu_pred, label='linear model')
# plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)

# compute posterior using RBF kernel
kern = GPy.kern.RBF(1, variance=1, lengthscale=.5)
kzx = kern.K(x, xsamp[:,None])
kzz = kern.K(x)
kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred, color='C1', label='RBF kernel')
plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1, color='C1')

# compute posterior using Mat32 kernel
kern = GPy.kern.Matern32(1, variance=1, lengthscale=.5)
kzx = kern.K(x, xsamp[:,None])
kzz = kern.K(x)
kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred, color='C2', label='Mat32 kernel')
plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1, color='C2')

# compute posterior using Mat52 kernel
# kern = GPy.kern.Matern52(1, variance=1, lengthscale=1)
# kzx = kern.K(x, xsamp[:,None])
# kzz = kern.K(x)
# kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

# mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
# cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
# std_pred = np.sqrt(np.diag(cov_pred))

# plt.plot(x, mu_pred, color='C3', label='Mat52 kernel')
# plt.fill_between(x[:,0], mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1, color='C3')

plt.scatter(xsamp, fsamp, color='k')
plt.legend()

plt.savefig('figures/f_linreg_condition_rbf_vs_mat.pdf', bbox_inches='tight')

# Optimization

In [ ]:
ls = 10**np.linspace(-1,2)


for var in 10**np.arange(-1,2,.5):
    loglike = []
    
    for i, l in enumerate(ls):
        kern = GPy.kern.RBF(1, variance=var, lengthscale=l)
        kxx = kern.K(xsamp[:,None]) + sigma*np.eye(xsamp.shape[0])

        loglike.append(scipy.stats.multivariate_normal.logpdf(np.zeros(xsamp.shape[0]), fsamp, kxx))

    plt.plot(ls, loglike, label='$\sigma^2 = %.1lf$'%var)

plt.semilogx()
plt.legend()
plt.title('$log(p(y))$',fontsize=20)
plt.xlabel('$\ell$', fontsize=20)

plt.savefig('figures/logpdf_vs_hyperparams.pdf',bbox_inches='tight')